# Analysis Set-up

In [1]:
using DrWatson
@quickactivate "CompositionalMLStudy"

using CairoMakie
using Dates
using OMOPCDMCohortCreator
using CSV
using DataFrames
using IPUMS
using LibPQ
using MLJ

import DBInterface:
    connect,
    execute

import DrWatson:
  datadir

import FunSQL:
    reflect,
    render

import IPUMS:
  load_ipums_extract,
  parse_ddi

import OMOPCDMCohortCreator:
    GenerateDatabaseDetails,
    GenerateTables

import OHDSICohortExpressions:
    translate,
    Model

## Data Configuration

### IPUMS Data Directory

In [2]:
# IPUMS Data Directory
IPUMS_DIR = datadir("exp_raw", "IPUMS")

# DDI Data Dictionary
DDI_FILE = "cps_00097.xml"

# IPUMS CPS Example Data 
DAT_FILE = "cps_00097.dat"

"cps_00097.dat"

### Load Data

In [3]:
omop_db_conn = connect(LibPQ.Connection, "user=thecedarprince dbname=synthea");
GenerateDatabaseDetails(:postgresql, "omop");
omop_tables = GenerateTables(omop_db_conn, exported = true);

DETAIL:  The database was created using collation version 2.37, but the operating system provides version 2.38.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE synthea REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
[ Info: Global database dialect set to: postgresql
[ Info: Global schema set to: omop
[ Info: source_to_concept_map table generated internally
[ Info: payer_plan_period table generated internally
[ Info: vocabulary table generated internally
[ Info: note_nlp table generated internally
[ Info: procedure_occurrence table generated internally
[ Info: domain table generated internally
[ Info: metadata table generated internally
[ Info: cohort_summary_stats table generated internally
[ Info: fact_relationship table generated internally
[ Info: drug_exposure table generated internally
[ Info: all_visits table generated internally
[ Info: final_visit_ids table generated internally
[ Info: person table

In [4]:
ddi = parse_ddi(joinpath(IPUMS_DIR, DDI_FILE));
ipums = load_ipums_extract(ddi, joinpath(IPUMS_DIR, DAT_FILE));

In [5]:
weather_data = CSV.read(datadir("exp_raw", "climdiv-tmpcst-v1.0.0-20241021-filtered"), DataFrame, header = false, delim = "  ")
filter!(row -> row.Column1[5:6] == "02", weather_data)
states = [row[1:3] for row in weather_data.Column1]
years = [parse(Int, row[7:10]) for row in weather_data.Column1]
weather_data = weather_data[:, Not(:Column1, :Column14)]
rename!(weather_data, Dates.monthname.(1:12))
weather_data.Year = years
weather_data.State = states

┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 4072. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/XLcqT/src/file.jl:592
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 4072. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/XLcqT/src/file.jl:592
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 4086. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/XLcqT/src/file.jl:592
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 4086. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/XLcqT/src/file.jl:592
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 6275. Filling remaining columns with `missing`
└ @ CSV ~/.julia/packages/CSV/XLcqT/src/file.jl:592
┌ Warning: thread = 1 warning: only found 13 / 14 columns around data row: 6284. Filling remaining columns with `missing`
└ @ CSV 

12614-element Vector{String31}:
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 "001"
 ⋮
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"
 "365"

# Patients with a History of Myocardial Infarction

## Finding Patients Who Have Had Myocardial Infarction

In [6]:
cohort_expression = read("data/myocardial_infarction.json", String)

fun_sql = translate(
    cohort_expression,
    cohort_definition_id = 1
);

catalog = reflect(
    omop_db_conn;
    schema = "omop",
    dialect=:postgresql
);

sql = render(catalog, fun_sql);

res = execute(omop_db_conn,
    """
    INSERT INTO
        omop.cohort
    SELECT
        *
    FROM
        ($sql) AS foo;
    """
)

cohort = GetCohortSubjects([1], omop_db_conn)

Row,cohort_definition_id,subject_id
,Int64?,Int64?
1,1,39
2,1,122
3,1,173
4,1,258
5,1,291
6,1,442
7,1,490
8,1,502
9,1,509


## Finding When Initial Myocardial Infarctions Took Place

In [7]:
cohort = GetCohortSubjectStartDate([1], cohort.subject_id, omop_db_conn)
rename!(cohort, :subject_id => :person_id)

Row,cohort_definition_id,person_id,cohort_start_date
,Int64?,Int64?,Date?
1,1,39,1998-12-12
2,1,122,1988-08-17
3,1,173,1976-01-06
4,1,258,1958-01-14
5,1,291,1982-06-17
6,1,442,1952-10-12
7,1,490,1978-12-27
8,1,502,1960-05-17
9,1,509,2003-02-19


## Getting Patient Demographic Information

In [8]:
cohort = GetPatientGender(cohort, omop_db_conn) |>
x -> GetPatientAgeGroup(x, omop_db_conn) |>
x -> GetPatientRace(x, omop_db_conn)

Row,person_id,race_concept_id,age_group,gender_concept_id,cohort_definition_id,cohort_start_date
,Int64?,Int32?,String?,Int32?,Int64?,Date?
1,39,8527,40 - 49,8507,1,1998-12-12
2,122,8516,40 - 49,8532,1,1988-08-17
3,173,8527,60 - 69,8507,1,1976-01-06
4,258,8527,60 - 69,8532,1,1958-01-14
5,291,8527,70 - 79,8532,1,1982-06-17
6,442,8527,80 - 89,8507,1,1952-10-12
7,490,8516,40 - 49,8532,1,1978-12-27
8,502,8527,70 - 79,8507,1,1960-05-17
9,509,8527,40 - 49,8507,1,2003-02-19


# Manually Combining Data Together

## Remarks

Inspecting the datasets, it seems like the following holds true:

- The temporal axis is the constraining feature between all these data sets.

    * The temporal limitation observed comes from the IPUMS data 

- The only reliably shared features between datasets are:

    * Year

    * Month

- Data can really only be used reliably between $2011$ and $2010$

## Filtering and Combining Datasets

In [9]:
filter!(row -> row.Year == 2011, weather_data)
ipums = ipums[!, Not([:CPSIDP, :ASECWT, :CPSID, :PERNUM, :ASECFLAG, :ASECWTH, :SERIAL, :YEAR])]
patients = filter!(row -> year(row.cohort_start_date) == 2011 || year(row.cohort_start_date) == 2010, cohort)

Row,person_id,race_concept_id,age_group,gender_concept_id,cohort_definition_id,cohort_start_date
,Int64?,Int32?,String?,Int32?,Int64?,Date?
1,8000,8527,10 - 19,8507,1,2010-06-24
2,26415,8527,10 - 19,8507,1,2010-06-30
3,33302,8527,20 - 29,8507,1,2010-03-31
4,35114,8527,20 - 29,8532,1,2010-06-11
5,43953,8527,20 - 29,8507,1,2010-07-29
6,48053,8515,30 - 39,8507,1,2010-04-19
7,52706,8527,30 - 39,8507,1,2010-08-21
8,56736,8527,40 - 49,8507,1,2011-02-16
9,90045,8527,10 - 19,8507,1,2011-07-26


## Creating DataFrame for Prediction